In [1]:
# General stuff
import os
import re
import time
import warnings
import math
import sys
import scipy.io as sio
import numpy as np
import matplotlib.pyplot as plt
import nibabel as nib
from subprocess import Popen, PIPE
from os.path import join as opj
from IPython.core.debugger import Tracer
from joblib import Parallel, delayed
from nilearn.decomposition import CanICA,DictLearning
from sklearn.decomposition import DictionaryLearning

# sklearn stuff
from sklearn.decomposition import TruncatedSVD,FastICA,SparsePCA,MiniBatchSparsePCA
from sklearn.utils.extmath import randomized_svd
from sklearn.linear_model import Ridge
from sklearn.decomposition import dict_learning_online
from sklearn.cross_decomposition import CCA
from sklearn.preprocessing import scale
import sklearn.metrics.cluster as metrics
from sklearn.cluster import AgglomerativeClustering
from sklearn.cluster.hierarchical import _hc_cut # Internal function to cut ward tree, helps speed up things a lot
from sklearn.utils import resample
from sklearn.model_selection import KFold

# personal functions
from importlib import reload
import fsutils as fs
reload(fs)

if os.path.exists('/data1/vbeliveau/'):
    # On NRU server
    cluster_code='/data1/vbeliveau/5HT_clustering'
    cluster_data='/data1/vbeliveau/5HT_clustering_data'
    atlas_dir='/data2/5HT_atlas'
    importPET=opj(atlas_dir,'import','PET')
    procPET=opj(atlas_dir,'proc','PET')
    subjects_dir=opj(atlas_dir,'proc','MR','recon_final')
elif os.path.exists('C:/Users/vbeliveau/'):
    # On laptop
    cluster_code='C:/Users/vbeliveau/Documents/GitHub/5HT_clustering'
    cluster_data='C:/Users/vbeliveau/Documents/5HT_clustering_data'
    subjects_dir='C:/Users/vbeliveau/Downloads/'
else:
    raise ValueError('Unknown location')
    
# Load local version of nilearn
if os.path.exists('/data1/vbeliveau/'):
    sys.path.append('/data1/vbeliveau/nilearn')
    import mynilearn.decomposition as dcm 
    reload(dcm)
elif os.path.exists('C:/Users/vbeliveau/'):
    sys.path.append('C:/Users/vbeliveau/Downloads/nilearn-master')
    import mynilearn.decomposition as dcm
    reload(dcm)
else:
    raise ValueError('Unknown location')
    
# Analysis directories
surf_data=opj(cluster_data,'surf_data')
fs.assert_dir(surf_data)
BPnd_data=opj(cluster_data,'BPnd')
fs.assert_dir(BPnd_data)
MFA_data=opj(cluster_data,'MFA')
MFA_preproc=opj(MFA_data,'preproc')
fs.assert_dir(MFA_data)
fs.assert_dir(MFA_preproc)

info_path=opj(cluster_code,'info_alltracers_base_healthy_hrrt.mat')
hemi_type=['lh','rh']

info=sio.loadmat(info_path)
tracers=['cumi','dasb','sb','az','C36']
petID=np.array([item for subl in info['petID'] for subsubl in subl for item in subsubl])
mrID=np.array([item for subl in info['mrID'] for subsubl in subl for item in subsubl])

In [ ]:
# Compute gradient structure for surfaces

# targ_list=['fsaverage5','fsaverage6','fsaverage']
targ_list=['fsaverage6']
surf_type='sphere'
hemi=['lh']
dest=opj(surf_data,'gradient')
fs.assert_dir(dest)

for targ in targ_list:
    for hemi in hemi_type:
        fname=opj(subjects_dir,targ,'surf',hemi+'.'+surf_type)
        fmask=opj(surf_data,'mask',targ+'.'+hemi)
        fout=opj(dest,targ+'.'+surf_type+'.'+hemi)
        fs.surf_gradient_struct(fname,fmask,verbose=True,save_out=fout)

Processing /data2/5HT_atlas/proc/MR/recon_final/fsaverage6/surf/lh.sphere
Extracting cortical vertices bordering medial wall
Computing gradient structure for cortical vertices
> /indirect/data1/vbeliveau/5HT_clustering/fsutils.py(503)surf_gradient_struct()
    501         Tracer()()
    502         # Find the normal of each face the current vertice is part of and take the average
--> 503         faces_ind=np.where(np.sum(faces==nc,axis=1)>=1)[0] # Extract rows for each faces containing the current vertice
    504         fnorm=np.ndarray([3,len(faces_ind)])
    505         for nf,nn in zip(faces_ind,np.arange(0,len(faces_ind))):



In [14]:
# Map all surface areas to fsaverage (might need to redo this if MSM is used)

tracers=['cumi','dasb','sb','az','C36']
hemi_type=['lh','rh']
ico_list=['6']

dest=opj(surf_data,'area')
fs.assert_dir(dest)
log_dest=opj(dest,'log')
fs.assert_dir(log_dest)

for ico in ico_list:
    for hemi in hemi_type:
        def mri_surf2surf(subj):
            log_file=open(opj(log_dest,'mri_surf2surf.'+subj+'.'+ hemi +'.log'),'w')
            sval=opj(atlas_dir,'proc','MR','recon_final',subj,'surf',hemi+'.area.mid')
            out=opj(dest,subj+'.ico'+ico+'.'+hemi+'.nii.gz')
            p=Popen(['mri_surf2surf','--hemi',hemi,'--srcsubject',subj,'--sval',sval,
                    '--trgsubject','ico','--trgicoorder',ico,'--trgsurfval',out],
                    stdout=log_file, stderr=log_file)
            p.communicate()
            log_file.close()
        mrID_list=[x+'_GD' for x in np.unique(mrID)]
        Parallel(n_jobs=20)(delayed (mri_surf2surf)(subj) for subj in mrID_list)

In [83]:
# Compute area for sphere

hemi_type=['lh','rh']
ico_list=['6']
for ico in ico_list:
    if ico==7:
        trg = 'fsaverage' 
    else:
        trg = 'fsaverage'+ico
    for hemi in hemi_type:
        mesh=opj(subjects_dir,trg,'surf',hemi+'.sphere')
        area=fs.surf_compute_area(mesh,verbose=True)
        fname=opj(surf_data,'area','ico'+ico+'.'+hemi+'.nii.gz')
        fmask=opj(surf_data,'mask',trg+'.'+hemi)
        fs.save_surf_data(area,fname,mask=fmask)

Processing /data2/5HT_atlas/proc/MR/recon_final/fsaverage6/surf/lh.sphere
5000/40962
10000/40962
15000/40962
20000/40962
25000/40962
30000/40962
35000/40962
40000/40962


/data1/vbeliveau/software/anaconda3/lib/python3.5/site-packages/nibabel/spatialimages.py:411: UserWarning: Using large vector Freesurfer hack; header will not be compatible with SPM or FSL
  hdr.set_data_shape(shape)


Processing /data2/5HT_atlas/proc/MR/recon_final/fsaverage6/surf/rh.sphere
5000/40962
10000/40962
15000/40962
20000/40962
25000/40962
30000/40962
35000/40962
40000/40962


In [136]:
# Compute gradient for BPnd

tracers=['cumi','dasb','sb','az','C36']
ico='6'
hemi_type=['lh','rh']
fwhm='5'
surf_type='sphere'
fgrad=opj(surf_data,'gradient',targ+'.'+surf_type+'.'+hemi)
bpnd_dest=opj(surf_data,'bpnd.mrtm2.nopvc')

# Process BPnd
grad_dest=opj(BPnd_data,'gradient')
fs.assert_dir(grad_dest)

if ico==7:
    trg = 'fsaverage' 
else:
    trg = 'fsaverage'+ico

for tracer in tracers:
    for hemi in hemi_type:
        petID_list=petID[np.array([re.search('^'+tracer,x) is not None for x in petID])]
        mrID_list=mrID[np.array([re.search('^'+tracer,x) is not None for x in petID])]
        mrID_list=[x+'_GD' for x in mrID_list]
        
        # Load PET data and average
        data=np.ndarray(len(petID_list),dtype=object)
        for subj,n in zip(petID_list,np.arange(0,len(petID_list))):
            fname=opj(bpnd_dest,subj+'.ico'+ico+'.'+hemi+'.sm'+fwhm,'bp.nii.gz')
            data[n]=fs.load_surf_data(fname)
        data=np.vstack(data).mean(axis=0)
        
        # Load area data and average
        area=np.ndarray(len(mrID_list),dtype=object)
        for subj,n in zip(mrID_list,np.arange(0,len(mrID_list))):
            fname=opj(surf_data,'area',subj+'.ico'+ico+'.'+hemi+'.nii.gz')
            area[n]=fs.load_surf_data(fname)
        area=np.vstack(area).mean(axis=0)
        
        # Load average surface area
        fname=opj(surf_data,'area','ico'+ico+'.'+hemi+'.nii.gz')
        avg_area=fs.load_surf_data(fname)
        
        # Compute area ratio
        fneigh=opj(surf_data,'neigh',trg+'.'+hemi+'.cortex.list')
        area_ratio=fs.surf_area_ratio(area,avg_area,fneigh)
        
        # Compute gradient        
        fgrad=opj(surf_data,'gradient',trg+'.'+surf_type+'.'+hemi)
        grad=fs.surf_gradient(data,fgrad,area_ratio=area_ratio,fneigh=fneigh)
        fname=opj(dest,'grad.'+tracer+'.ico'+ico+'.'+hemi+'nii.gz')
        fmask=opj(surf_data,'mask',trg+'.'+hemi)
        fs.save_surf_data(grad,fname,mask=fmask)

> /indirect/data1/vbeliveau/5HT_clustering/fsutils.py(572)surf_gradient()
    570 
    571     Tracer()()
--> 572     if area_ratio is not None:
    573         if verbose:
    574             print('Applying area correction')

ipdb> n
> /indirect/data1/vbeliveau/5HT_clustering/fsutils.py(573)surf_gradient()
    571     Tracer()()
    572     if area_ratio is not None:
--> 573         if verbose:
    574             print('Applying area correction')
    575         area_neigh,area_vertices=load_surf_neighborhood(fneigh)

ipdb> n
> /indirect/data1/vbeliveau/5HT_clustering/fsutils.py(575)surf_gradient()
    573         if verbose:
    574             print('Applying area correction')
--> 575         area_neigh,area_vertices=load_surf_neighborhood(fneigh)
    576         if area_ratio is not None:
    577             for nc in np.arange(0,len(cortex)):

ipdb> n
> /indirect/data1/vbeliveau/5HT_clustering/fsutils.py(576)surf_gradient()
    574             print('Applying area correction')
 

/indirect/data1/vbeliveau/5HT_clustering/fsutils.py:1: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 40961 but corresponding boolean dimension is 37476
  import numpy as np


array([array([ 0.3581608 ,  0.37917011,  0.38724795,  0.38235468,  0.37787507])], dtype=object)
ipdb> np.sum(ind)
1
ipdb> nb==area_neigh[ind]
array([False], dtype=bool)
ipdb> area_neigh[ind]
array([[9372, 9374, 9375, 9377, 9379]], dtype=object)
ipdb> nb
0
ipdb> neigh[nc]
array([    0, 10242, 10244, 10245, 10247, 10249])
ipdb> q


/indirect/data1/vbeliveau/5HT_clustering/fsutils.py:580: VisibleDeprecationWarning: boolean index did not match indexed array along dimension 0; dimension is 40961 but corresponding boolean dimension is 37476
  proj[nc][nb,:]=proj[nc][nb,:]*area_ratio[ind][nb==area_neigh[ind]]


ValueError: operands could not be broadcast together with shapes (2,) (0,) 

In [138]:
gs,_,_,_=fs.load_surf_gradient(fgrad)

In [147]:
neigh,vert=fs.load_surf_neighborhood(opj(surf_data,'neigh','fsaverage5.lh.list'))

In [148]:
neigh[0]

[0, 2562, 2564, 2565, 2567, 2569]